In [90]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler


In [80]:
long_list = pd.read_csv("../Datasets/sgemm_product_dataset/sgemm_product.csv",delimiter=',',nrows=1)
data = pd.read_csv("../Datasets/sgemm_product_dataset/sgemm_product.csv",delimiter = ',',names=long_list,skiprows=1)


In [127]:
X = pd.DataFrame(data,columns=['MWG','NWG','KWG','MDIMC','NDIMC','MDIMA','NDIMB','KWI','VWM','VWN','STRM','STRN','SA','SB'])
y = pd.DataFrame(data,columns=['Run1 (ms)','Run2 (ms)','Run3 (ms)','Run4 (ms)'])


X

,MWG,NWG,KWG,MDIMC,NDIMC,MDIMA,NDIMB,KWI,VWM,VWN,STRM,STRN,SA,SB
0,16,16,16,8,8,8,8,2,1,1,0,0,0,0
1,16,16,16,8,8,8,8,2,1,1,0,0,0,1
2,16,16,16,8,8,8,8,2,1,1,0,0,1,0
3,16,16,16,8,8,8,8,2,1,1,0,0,1,1
4,16,16,16,8,8,8,8,2,1,1,0,1,0,0
5,16,16,16,8,8,8,8,2,1,1,0,1,0,1
6,16,16,16,8,8,8,8,2,1,1,0,1,1,0
7,16,16,16,8,8,8,8,2,1,1,0,1,1,1
8,16,16,16,8,8,8,8,2,1,1,1,0,0,0
9,16,16,16,8,8,8,8,2,1,1,1,0,0,1


In [134]:
class RegressionModels():


  def scores(self,y_test,y_pred, model):
    print("Variance Score : " , explained_variance_score(y_test, y_pred))
    print("R2 Score : " ,r2_score(y_test,y_pred))
    print("Root Mean Square : ",math.sqrt(mean_squared_error(y_test, y_pred)))
    print("Best Parameters : ", model.best_params_ )

  def svmRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------SVM Regression Starts------------\n\n ")
    clf = SVR()
    svm_grid = GridSearchCV(clf, params, verbose=False, cv=3,return_train_score=True)
    svm_grid.fit(X_train,y_train)
    svm_predict = svm_grid.predict(X_test)
    print("Best Parameters : ", svm_grid.best_params_ )
    self.scores(Y_test, svm_predict, svm_grid)
    print("\n-----------SVM Regression ends------------\n\n ")

  def decisionTreeRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------Decision Tree Regression Starts------------\n\n")
    DTregressor = DecisionTreeRegressor(random_state=0)
    DT_grid = GridSearchCV(DTregressor, params, verbose=False, cv=3,return_train_score=True)
    DT_grid.fit(X_train,y_train)
    DT_predict = DT_grid.predict(X_test)
    self.scores(Y_test, DT_predict, DT_grid)
    print("\n-----------Decision Tree Regression Ends------------\n\n")

  def randomForestRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------Random Forest Regression Starts------------\n\n")
    RFRegressor = RandomForestRegressor(random_state=0)
    RF_grid = GridSearchCV(RFRegressor, params, verbose=False, cv=3, return_train_score=True)
    RF_grid.fit(X_train,y_train)
    RF_predict = RF_grid.predict(X_test)
    self.scores(Y_test, RF_predict, RF_grid)
    print("\n-----------Random Forest Regression Ends------------\n\n")
  
  def adaBoostRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------AdaBoost Regression Starts------------\n\n")
    AdaRegressor = AdaBoostRegressor(random_state=0)
    adaBoost_grid = GridSearchCV(AdaRegressor, params, verbose=False, cv=3,return_train_score=True)
    adaBoost_grid.fit(X_train,y_train)
    adaBoost_predict = adaBoost_grid.predict(X_test)
    self.scores(Y_test, adaBoost_predict, adaBoost_grid)
    print("\n-----------AdaBoost Regression Ends------------\n\n")

  def gaussianProcessRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------GaussianProcess Regression Starts------------\n\n")
    GPRRegressor = GaussianProcessRegressor(random_state=0)
    GPR_grid = GridSearchCV(GPRRegressor, params, verbose=False, cv=3,return_train_score=True)
    GPR_grid.fit(X_train,y_train)
    GPR_predict = GPR_grid.predict(X_test)
    self.scores(Y_test, GPR_predict, GPR_grid)
    print("\n-----------GaussianProcess Regression Ends------------\n\n")

  def LinearRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------Linear Regression Starts------------\n\n")
    LinearRegressor = LinearRegression()
    linearRegression_grid = GridSearchCV(LinearRegressor, params, verbose=False, cv=3,return_train_score=True)
    linearRegression_grid.fit(X_train,y_train)
    linearRegression_predict = linearRegression_grid.predict(X_test)
    self.scores(Y_test, linearRegression_predict, linearRegression_grid)
    print("\n-----------Linear Regression Ends------------\n\n")

  def mlpRegression(self, X_train, X_test, y_train, Y_test, params):
    print("-----------Neural Network Regression Starts------------\n\n")
    MLPRegressor_obj = MLPRegressor(random_state=0)
    MLPRegressor_grid = GridSearchCV(MLPRegressor_obj, params, verbose=False, cv=3,return_train_score=True)
    MLPRegressor_grid.fit(X_train,y_train)
    MLPRegressor_predict = MLPRegressor_grid.predict(X_test)
    self.scores(Y_test, MLPRegressor_predict, MLPRegressor_grid)
    print("\n-----------Neural Network Regression Ends------------\n\n")
    

  def train_split(self,X,y,test_size=0.2,random_state=0):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
    return X_train,X_test,y_train,y_test
 
  def train_all__models(self, X, y):
    svm_regression_params =  { 'C' : np.logspace(0, 3, 4), 'gamma' : np.logspace(-2, 1, 4)}
    dt_params = {'max_depth' : np.arange(1, 10, 10),'min_samples_split': np.arange(0.1, 1.0, 10)}
    rd_params = {'n_estimators' : np.arange(10,100,10),'max_depth' : np.arange(1,6,2)}
    ada_params = {'n_estimators' : np.arange(10,100,10)}
    gpr_params = {'n_restarts_optimizer' : np.arange(1,10,1)}
    linear_params = {'n_jobs' : np.arange(1,5,1)}
    mlp_params = {'hidden_layer_sizes': np.arange(30,150,20),'learning_rate': ['constant','invscaling','adaptive'],'max_iter': np.arange(20,200,50)}
    X_train,X_test,y_train,y_test = self.train_split(X,y)
    self.svmRegression(X_train,X_test,y_train,y_test, svm_regression_params)
    self.decisionTreeRegression(X_train,X_test,y_train,y_test, dt_params)
    self.randomForestRegression(X_train,X_test,y_train,y_test, rd_params)
    self.adaBoostRegression(X_train,X_test,y_train,y_test, ada_params)
    self.gaussianProcessRegression(X_train,X_test,y_train,y_test, gpr_params)
    self.LinearRegression(X_train,X_test,y_train,y_test, linear_params)
    self.mlpRegression(X_train,X_test,y_train,y_test, mlp_params)

  def wine_quality(self):
    df = pd.read_csv('../Datasets/Wine Quality/winequality-red.csv',delimiter=';')
    df.dropna(axis=0,inplace=True)
    #print(df.corr()['quality'].drop('quality'))
    X = df[df.columns[0:11]]
    y = df[df.columns[11:12]]
    self.train_all__models(X, y.values.ravel())

  def communities(self):
    columns_data = ['state','county','community','communityname','fold','population','householdsize','racepctblack','racePctWhite','racePctAsian','racePctHisp','agePct12t21',
                    'agePct12t29','agePct16t24','agePct65up','numbUrban','pctUrban','medIncome','pctWWage','pctWFarmSelf','pctWInvInc','pctWSocSec','pctWPubAsst','pctWRetire','medFamInc',
                    'perCapInc','whitePerCap','blackPerCap','indianPerCap','AsianPerCap','OtherPerCap','HispPerCap','NumUnderPov','PctPopUnderPov','PctLess9thGrade',
                    'PctNotHSGrad','PctBSorMore','PctUnemployed','PctEmploy','PctEmplManu','PctEmplProfServ','PctOccupManu','PctOccupMgmtProf','MalePctDivorce','MalePctNevMarr',
                    'FemalePctDiv','TotalPctDiv','PersPerFam','PctFam2Par','PctKids2Par','PctYoungKids2Par','PctTeen2Par','PctWorkMomYoungKids','PctWorkMom','NumIlleg','PctIlleg',
                    'NumImmig','PctImmigRecent','PctImmigRec5','PctImmigRec8','PctImmigRec10','PctRecentImmig','PctRecImmig5','PctRecImmig8','PctRecImmig10','PctSpeakEnglOnly','PctNotSpeakEnglWell'
                    ,'PctLargHouseFam','PctLargHouseOccup','PersPerOccupHous','PersPerOwnOccHous','PersPerRentOccHous','PctPersOwnOccup','PctPersDenseHous','PctHousLess3BR',
                    'MedNumBR','HousVacant','PctHousOccup','PctHousOwnOcc','PctVacantBoarded','PctVacMore6Mos','MedYrHousBuilt','PctHousNoPhone','PctWOFullPlumb','OwnOccLowQuart','OwnOccMedVal',
                    'OwnOccHiQuart','RentLowQ','RentMedian','RentHighQ','MedRent','MedRentPctHousInc','MedOwnCostPctInc','MedOwnCostPctIncNoMtg','NumInShelters','NumStreet','PctForeignBorn',
                    'PctBornSameState','PctSameHouse85','PctSameCity85','PctSameState85','LemasSwornFT','LemasSwFTPerPop','LemasSwFTFieldOps','LemasSwFTFieldPerPop','LemasTotalReq','LemasTotReqPerPop',
                    'PolicReqPerOffic','PolicPerPop','RacialMatchCommPol','PctPolicWhite','PctPolicBlack','PctPolicHisp','PctPolicAsian','PctPolicMinor','OfficAssgnDrugUnits','NumKindsDrugsSeiz',
                    'PolicAveOTWorked','LandArea','PopDens','PctUsePubTrans','PolicCars','PolicOperBudg','LemasPctPolicOnPatr','LemasGangUnitDeploy','LemasPctOfficDrugUn','PolicBudgPerPop',
                    'ViolentCrimesPerPop'
                    ]
    df = pd.read_csv('communities.data',delimiter=',',names=columns_data)

    #print("Before Removing the Non Predictable Features \n")
    #print("Shape Before Removing :" +str(df.shape) + "\n")
    #print(df.head())
    df = df.replace('?',np.nan)
    #print("After Removing the Non predictable Features \n ")
    #print("Shape After Removing :" + str(df.shape) + "\n")
    #print(df.head())
    print("Before Droping data Shape " + str(df.shape))
    #According to Dataset Description there are 5 non predictive features which can be removed
    df = df.drop(['fold','community','state','communityname','county'],axis=1)

    print("Checking the Columns Containing the null Values")
    
    #for i in range(0,120,41):
    #  print(df.iloc[:,i:i+41].isna().sum())
    #  print("\n")
    median_value = df.iloc[:,25].median(skipna = True)
    df.iloc[130,25] = median_value
    
    df = df.dropna(axis=1)
    print(df.columns)
    print("After Droping data Shape " + str(df.shape))
    #Replacing the columns with median
  

    print("Number of Missing Values in column is " + str(df.iloc[:,25].isna().sum()))
    print(df.shape)
    X = df[df.columns[0:100]]
    y = df[df.columns[100:101]]
    print(X.describe())
    #print(y)
    #print(X.head())
    self.train_all__models(X, y.values.ravel())


In [135]:
scalar = StandardScaler()
X = scalar.fit_transform(X)
# X = pd.DataFrame(X)
X.shape
X[0:200,:]

array([[-1.51675724, -1.51675724, -1.2109972 , ..., -1.        ,
        -1.        , -1.        ],
       [-1.51675724, -1.51675724, -1.2109972 , ..., -1.        ,
        -1.        ,  1.        ],
       [-1.51675724, -1.51675724, -1.2109972 , ..., -1.        ,
         1.        , -1.        ],
       ...,
       [-1.51675724, -1.51675724, -1.2109972 , ...,  1.        ,
        -1.        ,  1.        ],
       [-1.51675724, -1.51675724, -1.2109972 , ...,  1.        ,
         1.        , -1.        ],
       [-1.51675724, -1.51675724, -1.2109972 , ...,  1.        ,
         1.        ,  1.        ]])

In [133]:
regressionModels = RegressionModels()
regressionModels.train_all__models(X[0:200,:], y['Run1 (ms)'].iloc[0:200])



-----------SVM Regression Starts------------

 
Best Parameters :  {'C': 1000.0, 'gamma': 0.01}
Variance Score :  0.8273331900086451
R2 Score :  0.80757530847101
Root Mean Square :  6.61823286816622

-----------SVM Regression ends------------

 
-----------Decision Tree Regression Starts------------




/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Parameters :  {'max_depth': 1, 'min_samples_split': 0.1}
Variance Score :  0.4235744109555297
R2 Score :  0.4219005657222634
Root Mean Square :  11.471307372403068

-----------Decision Tree Regression Starts------------


-----------Random Forest Regression Starts------------




/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Variance Score :  0.9705828349524621
R2 Score :  0.9702668276256883
Root Mean Square :  2.601550549692804
Best Parameters :  {'max_depth': 5, 'n_estimators': 90}

-----------Random Forest Regression Ends------------


-----------AdaBoost Regression Starts------------


Variance Score :  0.8358264600809053
R2 Score :  0.8207407545120763
Root Mean Square :  6.38781642356451
Best Parameters :  {'n_estimators': 20}

-----------Random Forest Regression Ends------------


-----------GaussianProcess Regression Starts------------


Variance Score :  0.7353508774084289
R2 Score :  0.6953551513499836
Root Mean Square :  8.327383469521987


/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/gursimransingh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


AttributeError: 'GridSearchCV' object has no attribute 'GPR_grid'